In [1]:
import numpy as np

%run -i runup_funcs.py
%run -i wave_funcs.py 


print(calc_L0(10))

print(func_list)

for item in func_list:
    cmd = "print({}())".format(item, item)
    print(item)
    exec(cmd)

print('Raub')
R2R, S, setup, Sinc, SIG, ir, R16R = calcR2_Raubenheimer(2., 10., .05, .005)
print(R2R)

156.13099917314932
['A17', 'D20', 'H86', 'NH91', 'P14', 'P16', 'R01', 'S06', 'S11', 'V12']
A17
0.3498845445000393
D20
5.574916526297034
H86
0.6933375474091239
NH91
1.764186045187383
P14
0.7407072495843278
P16
0.9333809511662428
R01
0.6747429110409386
S06
0.7371541008103424
S11
1.4210386883731967
V12
0.2105119278636574
Raub
1.0514324637448569


In [2]:
# SS band
f_lo = 0.04
f_hi = 0.25
T_lo = 1/f_lo
T_hi = 1/f_hi
w_lo = 2.*np.pi*f_lo
w_hi = 2.*np.pi*f_hi
print('T:',T_lo, T_hi)
print('f:',f_lo, f_hi)
print('w:',w_lo, w_hi)

T: 25.0 4.0
f: 0.04 0.25
w: 0.25132741228718347 1.5707963267948966


In [33]:
def jonswap_ess( Hs, Tp, flo = 0.04, fhi = 0.25, npts = 21, acc = 0.05):
    # Generate a Jonswap spectrum in sea-swell band for Lange et al
    # Input:
    #   Hs - Significant wave height (m)
    #   TP - Peak period (s)
    #   flo - Low-frequency cutoff (Hz) [0.04]
    #   fhi - High-frequency cutoff (Hz) [0.25]
    #   npts- number of points [21]
    #   acc - required accuracy [0.02]
    # Returns
    #   Ess - Energy spectrum (m2/Hz)
    #   f   - Frequency (Hz)
    #   df  - delta f (Hz)
    w = np.geomspace(2.*np.pi*flo, 2*np.pi*fhi, npts)
    Ess = 2.*np.pi*jonswap(w, Hs, Tp)
    f = w/(2.*np.pi)
    # check integration
    dff = np.diff(f)
    df = np.append(dff[0], dff)
    Hsf = 4.*np.sqrt( np.sum(Ess*df))
    err = np.abs(Hsf-Hs)/Hs
    print(Hsf, err)
    assert err <= acc,'Hs does not match input'
    return Ess, f, df

In [31]:
def two_slope_IPA_free(Ess, f, df, beta_ef, beta_eff):
    """
    Equation 9 in Lange et al., 2020
    """
    #term1 = 0.4 * 𝛽eff^0.45 ∫𝑆𝑆 𝐸^0.5 𝑓^−1.45 𝑑𝑓
    aint = Ess**0.5 * f**-1.45 * df
    term1 = 0.4 * beta_eff**0.45 * aint.sum()

    #term2 = 3.72 * 𝛽eff^0.6 ∫𝑆𝑆 𝐸^0.95 𝑓^−0.25 𝑑𝑓
    bint = Ess**0.95 * f**-0.25 * df
    term2= 3.72 * beta_eff**0.6 * bint.sum()

    #term3 = 0.76 * 𝛽f^2 * 𝛽eff^0.75 ∫𝑆𝑆 𝐸^0.4 𝑓^−3.1 𝑑𝑓
    cint = Ess**0.4 * f**-3.1 * df
    term3= 0.76 * beta_f**2 * beta_eff**0.75 * cint.sum()
    #term3.append(c)

    #R2%,G = term1 + 2[term2 + term3]^0.5
    R2 = term1 + 2*(term2+term3)**0.5
    return R2

In [34]:
beta_f = 0.05
beta_eff = 0.005
Ess, f, df = jonswap_ess(2., 10.)
R2IPA = two_slope_IPA_free(Ess, f, df, beta_f, beta_eff)
print(R2IPA)

2.0073711033839174 0.0036855516919587217
0.672110644099395
